In [ ]:
%set_env SHELL=/bin/bash


In [ ]:
import mbuild
compound = mbuild.load("spce216.gro")
compound.visualize()

Let us begin setting up our simulation files. Our goal is to simulate a box of water molecules, and we want the box to have at least 4000 water molecules as we have ascertained this is a reasonable system size.

**Question** How do we decide what is an optimal system size for any of our simulations?

We will first need to see what files we have, and what files do we need. 

In gromacs, input files are given in three main types:
(1) File that has information about the atom (atomtypes), and molecules in our system, and how they interact with each other -- Topology file (extension used is \*.top)
(2) A file with initial coordinates of all the atoms in our system. This is called the coordinate file and can be input in different formats. Most common are gro and pdb. We will use \*.gro format.
(3) A file with parameters for our MD simulations. We will see this file in detail later. The extension for this file is \*.mdp.

Note that every MD software has its own format and names for the input files. However, in the end, we need to provide these three pieces of information -- what does our system comprise, initial coordinates, and parameters for our MD simulation.

It is important to choose and understand each of these input carefully. 

**Question** Where do we get information about the atomtypes and interactions to give as input in our files? What does this look like? (Hint: Force fields)

In [ ]:
%%bash
pwd
ls 

We have a 
(1) topology file (spce_MASTER.top)
(2) two mdp files (em.mdp and mdspce.top)
(3) one gro file (spce216.gro)

Let's do the easy bit first. In the gro file, you will see we have 216 water molecules. However, we want a simulation box with 4000 water molecules. Therefore, we will run the following commands to generate a box of 4000 molecules. (Remember, these questions are gromacs specific and you will need different commands for different MD software but the general steps will be the same).

In [ ]:
!mpirun gmx_mpi solvate -cs spce216.gro -cp spce216.gro -box 5.0 -o spcesolv.gro

In [ ]:
import mbuild
compound = mbuild.load("spcesolv.gro")
compound.visualize()

That was easy! But let's understand what we did.

mpirun gmx_mpi_d solvate -cs spce216.gro -cp spce216.gro -box 5.0 -o spcesolv.gro

(1) gmx_mpi_d == invokes gromacs and the exact form of this gmx_(suffix) depends on how gromacs is installed and what is given as the suffix at the time of installation.
(2) solvate == is the gromacs command to solvate a solute (given as -cs option) with a solvent (given as -cp option)
(3) the command above is saying "solvate my solute (spce216.gro) with solvent (spce216.gro)"
(4) write the output as spcesolv.gro.

In [ ]:
!ls -ltrh

Ok, now let's understand our topology file. Remember I had asked a question -- how do we decide on our atomtypes and interactions between them? It comes from force fields. But at a very basic level, we have to have a way of saying to the computer, hey this coordinate you see is an oxygen and this one is a hydrogen, one oxygen atom is bonded to two hydrogen atoms. How do we convey this information?

**In gromacs topology file** you first define the atoms in your system, then the molecules, and then the system. So if you open your topology file you will see sections [atoms], [molecules], [system].


So now that you know what is in the topology file, let's think of what modifications, if any, do we need to make with our steps of creating a new box of water molecules.

**Question** What do we need to modify in the topology file to make it compatible with our configuration file?

In [ ]:
%%bash
fsolv="spcesolv.gro" 
nsol=`tail -2 $fsolv | head -1 | awk '{print substr($1,1,length($1)-3)}'`
sed  "s/XXXX/$nsol/g" spce_MASTER.top > spce-updated.top
tail -2 spce-updated.top

Ok, so now we have a configuration file. We have an updated topology file. We know what goes into it. **What is the next file we should be thinking about?"** 

Yes, it's the mdp file. Here we will enter all the parameters necessary to run the simulation. Let's think about this for a moment. **Do you know what is the general algorithm of an MD simulation?**

**What is the general approach when starting the simulations**
1. You run an energy minimization
2. Run an equilibration
3. Run a production simulation

We will specify minimization algorithm in the mdp file. 

In [ ]:
%%bash
more em.mdp

In [ ]:
!mpirun gmx_mpi grompp -f em.mdp -c spcesolv.gro -p spce-updated.top -o minim.tpr

In [ ]:
!mpirun gmx_mpi mdrun -s minim.tpr -deffnm spceminim -v

In [ ]:
import mbuild
compound = mbuild.load("spceminim.gro")
compound.visualize()

In [ ]:
%%bash
ls -ltrh

Now that we understand the input let's spend few minutes going over the mdp file and let me know if you have any questions.

Now let's compile and run the simulation!

In [ ]:
!mpirun gmx_mpi grompp -f runmdshort.mdp -c spceminim.gro -p spce-updated.top -o spcemd.tpr -maxwarn 1

In [ ]:
!mpirun gmx_mpi mdrun -s spcemd.tpr -deffnm spcemdshort -v 